In [1]:
import torch
from torch import nn
import numpy as np
from torch.utils.data import DataLoader
from datasets import load_dataset, Dataset, load_from_disk
from TTS.api import TTS
from torch.utils.data import DataLoader

from IPython.display import Audio
from copy import deepcopy
from tqdm.notebook import tqdm

device = "cuda" if torch.cuda.is_available() else "cpu"

In [24]:
next(iter(cv_13_shuffled))

Reading metadata...: 1013968it [00:16, 60557.91it/s]


{'client_id': 'fed04ddc1a9e44059bb6aca6eeceb50210050d152ed4d77696d45f239f3bfcbb2b64ed0916ca1977c78df826c252215a2ff2f80481737d27d8c09af53d27c1bf',
 'path': 'en_train_7/common_voice_en_19319553.mp3',
 'audio': {'path': 'en_train_7/common_voice_en_19319553.mp3',
  'array': array([ 0.00000000e+00,  3.97050556e-14, -4.55790936e-14, ...,
          1.06980515e-04,  1.36692775e-04,  1.38366246e-04]),
  'sampling_rate': 48000},
 'sentence': 'The publisher of the first three issues was Black Swan Press.',
 'up_votes': 3,
 'down_votes': 0,
 'age': 'thirties',
 'gender': 'male',
 'accent': 'England English',
 'locale': 'en',
 'segment': '',
 'variant': ''}

In [30]:
import scipy.io.wavfile as wavf
from copy import deepcopy

cv_13 = load_dataset("mozilla-foundation/common_voice_13_0", "en", split="train", streaming=True, trust_remote_code=True)
cv_13_shuffled = cv_13.shuffle(buffer_size=10_000, seed=42)

PATH = '../benchmark/commonvoice/audio/'

n = 100
bechmark_list = []
for i, item in enumerate(cv_13_shuffled):
    item_copy = deepcopy(item)
    file = item['path'].split('/')[-1]
    wavf.write(PATH+file, item['audio']['sampling_rate'], item['audio']['array'])

    item_copy['path'] = PATH+file
    bechmark_list.append(item_copy)
    if i >= n:
        break
bechmark_dataset = Dataset.from_list(bechmark_list)
bechmark_dataset.save_to_disk("../benchmark/commonvoice")

Reading metadata...: 1013968it [00:16, 61645.61it/s]


Saving the dataset (0/1 shards):   0%|          | 0/101 [00:00<?, ? examples/s]

In [7]:
import os

texts = ["Building a wall on the U.S.-Mexico border will take literally years.",
         "Wisconsin is on pace to double the number of layoffs this year.",
         "With its easily transported, thick-skinned and sweet-tasting fruit, the Gros Michel banana plant dominated the plantations of Central America",
         "Growing practices in South East Asia haven’t helped matters.",
         "Sleeping only seven hours a night, Kilian Jornet seems almost superhuman.",
         "For more than two hundred years the pessimists have been winning the public debate.",
         "Says that Tennessee law requires that schools receive half of proceeds -- $31 million per year -- from a half-cent increase in the Shelby County sales tax.",
         "We know that more than half of Hillary Clintons meetings while she was secretary of state were given to major contributors to the Clinton Foundation.",
         "In an attempt to inject some positivity into their feedback, many managers rely on sandwiching negative feedback between two positive comments.",
         "But according to the authors, the world isn't as bad as we think.",
         "Says the unemployment rate for college graduates is 4.4 percent and over 10 percent for noncollege-educated.",
         "Each year, 18,000 people die in America because they don't have health care."]

audios = ['original_audio0.wav',
 'original_audio1.wav',
 'original_audio2.wav',
 'original_audio3.wav',
 'original_audio4.wav',
 'original_audio5.wav',
 'original_audio6.wav',
 'original_audio7.wav',
 'original_audio8.wav',
 'original_audio9.wav',
 'original_audio10.wav',
 'original_audio11.wav',]


In [2]:
# model text-to-seach

tts = TTS("tts_models/multilingual/multi-dataset/xtts_v2").to(device)

pretrained_speaker_embedding = [speaker['speaker_embedding'].numpy().flatten() for speaker in tts.synthesizer.tts_model.speaker_manager.speakers.values()]
mean_embedding = torch.tensor(np.mean(pretrained_speaker_embedding, axis=0)).reshape(1, 512)
std_embedding = torch.tensor(pretrained_speaker_embedding).std(dim=0).reshape(1, 512)

del tts
torch.cuda.empty_cache()
        
class ConstantWrapper(nn.Module):
    def __init__(self, constant):
        super().__init__()
        self.constant = constant.to(device)
    
    def forward(self, X):
        return self.constant

class AddWrapper(nn.Module):
    def __init__(self, constant):
        super().__init__()
        self.constant = constant.to(device)
    
    def forward(self, X):
        return X + self.constant

class SpeakerEncoderWrapper(nn.Module):
    def __init__(self, speaker_encoder, type_of_augmentation=None, **kwargs):
        super().__init__()
        self.speaker_encoder = speaker_encoder
        self.aug_type = type_of_augmentation
        if self.aug_type:
            if self.aug_type == 'mean':
                self.correct_embeding = ConstantWrapper(mean_embedding)
            if self.aug_type == 'noise':
                noise = torch.normal(torch.zeros_like(mean_embedding), std_embedding)
                self.correct_embeding = AddWrapper(noise)
            if self.aug_type == 'normal':
                emb = torch.normal(mean_embedding, std_embedding)
                self.correct_embeding = ConstantWrapper(emb)
            if self.aug_type == 'random_embedding':
                idx = np.random.randint(0, len(pretrained_speaker_embedding))
                emb = torch.tensor(pretrained_speaker_embedding[idx]).reshape(1, 512)
                self.correct_embeding = ConstantWrapper(emb)
            if self.aug_type == 'combine_embeddings':
                self.alpha = kwargs['alpha']
                self.correct_embeding = ConstantWrapper(mean_embedding)
        else:
            self.correct_embeding = nn.Identity()

    def forward(self, x, l2_norm=False):
        out = self.speaker_encoder( x, l2_norm)
        if self.aug_type == 'combine_embeddings':
            corrected = self.alpha * self.correct_embeding(out) + (1 - self.alpha) * out
        else:
            corrected = self.correct_embeding(out)
        return corrected

 > tts_models/multilingual/multi-dataset/xtts_v2 is already downloaded.
 > Using model: xtts


/tmp/ipykernel_398325/670210822.py:7: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at ../torch/csrc/utils/tensor_new.cpp:275.)
  std_embedding = torch.tensor(pretrained_speaker_embedding).std(dim=0).reshape(1, 512)


In [3]:
import os
    
class SpeechAnonymizer(nn.Module):
    OUTPUT_DIR = '../output/audio/'
        
    def __init__(self, anonymize=False, augment=None, **kwargs):
        super().__init__()

        tts = TTS("tts_models/multilingual/multi-dataset/xtts_v2").to(device)
        model_silero, decoder, utils = torch.hub.load(repo_or_dir='snakers4/silero-models',
                                       model='silero_stt',
                                       language='en',
                                       device=device)
        read_batch, _, _, prepare_model_input = utils

        self.speech_to_text = model_silero
        self.text_decoder = decoder
        self.text_to_speech = tts
                    
        self._prepeare_model_inputs = prepare_model_input
        self._read_batch = read_batch

        self.speech_to_text.eval()
        self.text_to_speech.eval()
        
        self.augment = augment if augment else 'off'
        self.anonymize = anonymize
        if self.anonymize:
            (
                self
                .text_to_speech
                .synthesizer
                .tts_model
                .hifigan_decoder
                .speaker_encoder
            ) = SpeakerEncoderWrapper(
                (
                    self
                    .text_to_speech
                    .synthesizer
                    .tts_model
                    .hifigan_decoder
                    .speaker_encoder
                ),
                type_of_augmentation=augment, 
                alpha=kwargs.get('alpha')
            )


        self.OUTPUT_DIR += (self.augment + '/')
        if kwargs.get('alpha'):
            self.OUTPUT_DIR += (str(kwargs.get('alpha')) + '/')
        if not os.path.exists(self.OUTPUT_DIR[:-1]):
            os.makedirs(self.OUTPUT_DIR[:-1])
            

    def forward(self, data):
        file_path = data['path']
        output_path = self._get_output_path(file_path)

        # speach-to-text
        stt_input = self._prepeare_model_inputs(self._read_batch([file_path]), device=device)
        stt_output = self.speech_to_text(stt_input)
        text = self.text_decoder(stt_output[0].cpu())

        # text-to-speach
        self.text_to_speech.tts_to_file(text, speaker_wav=file_path, language="en", file_path=output_path)
        return output_path
        

    def _get_output_path(self, file_path):
        file_name = file_path.split('/')[-1]
        return self.OUTPUT_DIR + file_name
        

In [4]:
def show_audio(original_file, anonimized_file):
    print(original_file['sentence'])
    print('Original file')
    display(Audio(original_file['path']))
    print('Anonimized file')
    display(Audio(anonimized_file))

In [5]:
from transformers import AutoModelForSpeechSeq2Seq, AutoProcessor, pipeline

device = "cuda" if torch.cuda.is_available() else "cpu"
torch_dtype = torch.float16 if torch.cuda.is_available() else torch.float32

model_id = "openai/whisper-large-v3"

model = AutoModelForSpeechSeq2Seq.from_pretrained(
    model_id, torch_dtype=torch_dtype,  use_safetensors=True
)
model.to(device)

processor = AutoProcessor.from_pretrained(model_id)

pipe_stt = pipeline(
    "automatic-speech-recognition",
    model=model,
    tokenizer=processor.tokenizer,
    feature_extractor=processor.feature_extractor,
    max_new_tokens=128,
    chunk_length_s=30,
    batch_size=16,
    return_timestamps=True,
    torch_dtype=torch_dtype,
    device=device,
)

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [6]:
from jiwer import wer

In [13]:
import scipy.io.wavfile as wavf

embendings = torch.hub.load('RF5/simple-speaker-embedding', 'convgru_embedder')
embendings.eval();

def get_emb(path):
    _, wav = wavf.read(path)
    wav = torch.from_numpy(wav).float()
    return embendings(wav[None])

Using cache found in /home/ansafronov/.cache/torch/hub/RF5_simple-speaker-embedding_master


In [14]:
from torch.nn import CosineSimilarity
import pandas as pd


path_to_dataset = '../benchmark/commonvoice'

def get_stats_nn(model):
    input_data = load_from_disk(path_to_dataset)

    output_data = []
    for idx, inp in enumerate(tqdm(input_data)):
        in_path = inp['path']
        out_path = model(inp)

        emb_in = get_emb(in_path)
        emb_out = get_emb(out_path)

        text_reconstructed = pipe_stt(out_path)['text']
        text = inp['sentence']

        output_data.append({
            'cossim': CosineSimilarity()(emb_in, emb_out).cpu()[0].item(),
            'wer': wer(text, text_reconstructed)
        })
            
    data = pd.DataFrame(output_data)
    return {
        'cossim_mean': data['cossim'].mean(),
        'cossim_std': data['cossim'].std(ddof=1),
        'wer_mean': data['wer'].mean(),
        'wer_std': data['wer'].std(ddof=1)
    }

In [16]:
import json

with open('../output/results_nn.json', 'w') as f:
    json.dump(results_nn, f)

In [15]:
augments = [
    'off', 
    'mean', 
    'noise',
    'normal',
    'random_embedding',
    'combine_embeddings', 
]

params = {
    'combine_embeddings': [0.3, 0.5, 0.7]
}
results_nn = dict()
for augment in augments:
    print(augment)
    torch.cuda.empty_cache()
    if augment == 'off':
        model = SpeechAnonymizer(anonymize=False)
    elif augment in params:
        for param in params[augment]:
            model = SpeechAnonymizer(anonymize=True, augment=augment, alpha=param)
            results_nn[augment + '_' + str(param)] = get_stats_nn(model)
    else:
        model = SpeechAnonymizer(anonymize=True, augment=augment)

    if augment not in params:
        results_nn[augment] = get_stats_nn(model)
   


off
 > tts_models/multilingual/multi-dataset/xtts_v2 is already downloaded.
 > Using model: xtts


Using cache found in /home/ansafronov/.cache/torch/hub/snakers4_silero-models_master


  0%|          | 0/101 [00:00<?, ?it/s]

 > Text splitted to sentences.
['the publisher of the first three issues was black swan press']
 > Processing time: 1.3824613094329834
 > Real-time factor: 0.28212712750811936


Due to a bug fix in https://github.com/huggingface/transformers/pull/28687 transcription using a multilingual Whisper will default to language detection followed by transcription instead of translation to English.This might be a breaking change for your use case. If you want to instead always translate your audio to English, make sure to pass `language='en'`.


...

/home/ansafronov/Yandex.Disk/Studies/ml/ml_lpc_anonymisation/env/lib/python3.10/site-packages/transformers/pipelines/base.py:1157: UserWarning: You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset
  warnings.warn(


 > Text splitted to sentences.
['later the filer used was industrial sflag']
 > Processing time: 2.1372904777526855
 > Real-time factor: 0.29593624431356574


/home/ansafronov/Yandex.Disk/Studies/ml/ml_lpc_anonymisation/env/lib/python3.10/site-packages/transformers/pipelines/base.py:1157: UserWarning: You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset
  warnings.warn(


 > Text splitted to sentences.
['more causes are in there ed to be low']
 > Processing time: 1.6688988208770752
 > Real-time factor: 0.2951303974748132


/home/ansafronov/Yandex.Disk/Studies/ml/ml_lpc_anonymisation/env/lib/python3.10/site-packages/transformers/pipelines/base.py:1157: UserWarning: You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset
  warnings.warn(


In [39]:
import pandas as pd

In [46]:
data = pd.DataFrame(data)
data

,file,cossim,wer
0,original_audio0.wav,0.472850,0.000000
1,original_audio1.wav,0.263982,0.000000
2,original_audio2.wav,0.355806,0.263158
3,original_audio3.wav,0.557097,0.444444
4,original_audio4.wav,0.515755,0.181818
5,original_audio5.wav,0.367766,0.214286
6,original_audio6.wav,0.155975,0.259259
7,original_audio7.wav,0.373213,0.120000
8,original_audio8.wav,0.224443,0.000000
9,original_audio9.wav,0.474429,0.076923


In [47]:
data.mean()

/tmp/ipykernel_107898/531903386.py:1: FutureWarning: The default value of numeric_only in DataFrame.mean is deprecated. In a future version, it will default to False. In addition, specifying 'numeric_only=None' is deprecated. Select only valid columns or specify the value of numeric_only to silence this warning.
  data.mean()


cossim    0.388480
wer       0.167651
dtype: float64

In [107]:
%timeit model({'path': audio_path})

 > Text splitted to sentences.
["each year people die in america because they don't have health care"]
 > Processing time: 1.4592609405517578
 > Real-time factor: 0.2780373266553147
 > Text splitted to sentences.
["each year people die in america because they don't have health care"]
 > Processing time: 1.4762356281280518
 > Real-time factor: 0.28127156435973616
 > Text splitted to sentences.
["each year people die in america because they don't have health care"]
 > Processing time: 1.5124495029449463
 > Real-time factor: 0.28011617675661926
 > Text splitted to sentences.
["each year people die in america because they don't have health care"]
 > Processing time: 1.5323259830474854
 > Real-time factor: 0.27840959069048327
 > Text splitted to sentences.
["each year people die in america because they don't have health care"]
 > Processing time: 1.4959053993225098
 > Real-time factor: 0.27705209359512617
 > Text splitted to sentences.
["each year people die in america because they don't ha